Code for crawling Diabetes daily
- started with Type-1 diabetes forum
- experienced occasional timing-out (unobserved in other communities)

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# import requests, re, csv
# import urllib
# from urllib.request import Request,urlopen
# from TorCrawler import TorCrawler

import requests
from time import sleep
import time
import random
from bs4 import BeautifulSoup as BS
from fake_useragent import UserAgent
from lxml.html import fromstring
import numpy as np
import pickle
import os

In [ ]:
ua = UserAgent()

In [4]:
"""
I got too bothered to create a proper code for obtaining the pages of each forum and instead 
dumped all of them into this list    
"""
base_list = [
    ('feedback-forum',63),
    ('announcements', 19),
    ('diabetes-news-and-studies', 202),
    ('personal-updates', 45),
    ('type-1-diabetes', 551),
    ('type-1-5-diabetes', 40),
    ('type-2-diabetes', 1078),
    ('pre-diabetes', 104),
    ('pregnancy', 28),
    ('friends-and-family', 20),
    ('food-and-diet', 248),
    ('low-carb-diet', 81),
    ('recipes', 110),
    ('testing-blood-sugar', 154),
    ('multiple-daily-injections-mdi-', 47),
    ('insulin-pumps', 156),
    ('continuous-glucose-monitors-cgms-', 41),
    ('exercise', 66),
    ('weight-loss', 33),
    ('depression-and-staying-positive', 56),
    ('eyes', 24),
    ('kidneys', 10),
    ('skin-wound-and-infections', 2),
    ('heart', 7),
    ('neuropathy', 22),
    ('complications', 25),
    ('women-s-corner', 12),
    ('men-s-corner', 8),
    ('non-traditional-treatments', 21),
    ('united-states', 28),
    ('united-kingdom', 2),
    ('canada', 3),
    ('australia', 2),
    ('arts-and-culture', 7),
    ('research-and-clinical-trials', 26),
    ('other-medical-conditions', 48),
    ('promotions-surveys-and-trial-recruitment', 25),
    ('there-s-more-to-life-than-diabetes-', 273),
    ('humor', 106),
    ('non-diabetes-news-and-links', 32),
    ('diabetes-daily-meetups', 16),
    ('book-club', 9)
]

In [ ]:
# 2. open url with random agent
"""
1. Enters each forum, starts from the 1st page
2. For each page, crawls the
    (title, post address, username, user address(URL), date, number of replies, number of views)
    of each post
3. After each forum is completed, the data is stored in /data/site_name/forum_name/users.pckl as a list
"""

for k,(forum,NUM_PAGES) in enumerate(base_list):
    print('[%d] Crawling %s...' %(k+1,forum))
    start = time.time()
    base_url = 'https://www.diabetesdaily.com/forum/%s/' %forum
    if not os.path.exists(os.path.join('../data/diabetes_daily',forum)):
        os.mkdir(os.path.join('diabetes_daily',forum))

    outputs = []
    for page_num in range(1,NUM_PAGES+1):
        url = base_url + 'index%d?daysprune=-1'%(page_num)
        # start a request with a random agent
        sleep(np.random.randint(3,15))
        req = requests.get(url=url,headers={'User-Agent':ua.random})
        soup = BS(req.text,'lxml')
        S1 = soup.find('ol','threads')
        for S2 in S1.find_all(recursive=False):
            if 'moved' in S2['class']:
                continue
            else:
                title = S2.find('a','title').text.strip()
                post_addr = S2.find('a','title')['href'][len(base_url):-1].split('/')
                post_addr = '/'.join(post_addr[:-1])
                user_name = S2.find('a','username understate').text.strip()
                user_addr = S2.find('a','username understate')['href'].split('/')[-2]
                date_time = S2.find('a','username understate')['title'].split(' on ')[-1]
                scores = S2.find('ul','threadstats td alt').find_all(recursive=False)
                replies = int(scores[0].text.replace(',','').strip().split('\t')[-1])
                views = int(scores[1].text.replace(',','').strip().split(' ')[-1])
                outputs.append((title,post_addr,user_name,user_addr,date_time,replies,views))
        if (page_num)%50==0:
            print("Taking a 30 second rest at page %d..."%(page_num))
            sleep(30)
        with open('../data/diabetes_daily/%s/users.pckl'%forum,'wb') as f:
            pickle.dump(outputs,f)
    print('[%d] Crawled %s! %d users crawled' %(k+1,forum,len(outputs)))
    print(time.time()-start)

In [ ]:
# test: for obtaining the data of a single instance
# base_url = 'https://www.diabetesdaily.com/forum/introduce-yourself/'
# url = base_url+'index11?daysprune=-1' %
# req = requests.get(url=url,headers={'User-Agent':ua.random})
# soup = BS(req.text,'lxml')
# S1 = soup.find('ol','threads')
# for S2 in S1.find_all(recursive=False):
#     if 'moved' not in S2['class']:
#         title = S2.find('a','title').text.strip()
#         post_addr = S2.find('a','title')['href'][len(base_url):-1].split('/')
#         post_addr = '/'.join(post_addr[:-1])
#         user_name = S2.find('a','username understate').text.strip()
#         user_addr = S2.find('a','username understate')['href'].split('/')[-2]
#         date_time = S2.find('a','username understate')['title'].split(' on ')[-1]
#         scores = S2.find('ul','threadstats td alt').find_all(recursive=False)
#         replies = int(scores[0].text.replace(',','').strip().split('\t')[-1])
#         views = int(scores[1].text.replace(',','').strip().split(' ')[-1])
#         print(title)
#         print(post_addr)
#         print(user_name)
#         print(user_addr)
#         print(date_time)
#         print(replies)
#         print(views)
#         print('\n')